# Get config jsons from fine-tuning experiments and analyze

## Ny kjøring i 2024


In [49]:
import json, os, sys, io
from pathlib import Path
import pandas as pd
from datetime import date
import zipfile


In [50]:
# scp -r ec-egilron@fox.educloud.no:/fp/homes01/u01/ec-egilron/sqlabel-github/0816_all-logs.zip tsa-0816
# 08231629_configs.zip

In [51]:
configs_path = Path("/home/egil/gits_wsl/seq-label-github/configs/lumi")

name_filter = "01121628"
jsons = [f for f in configs_path.iterdir() if f.name.startswith(name_filter) and  f.name.endswith("json")]
len(jsons)


18

In [52]:

records = []
root_keys = ["timestamp",  "task",  "machinery",  "best_epoch" ]
args_keys = [ 'model_name_or_path', 'task_name', 'dataset_name', 'output_dir', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
epoch_keys = ['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1', 'eval_accuracy','epoch', 'step' ]
for j in jsons:
    jdata = json.loads(j.read_text())
    for epoch_data in jdata["train_epochs_val"]:
        record = {k:v for k,v in jdata.items() if k in root_keys}
        record["config-file"] = j.name
        record.update({k:v for k,v in jdata["args_dict"].items() if k in args_keys})
        record.update({k:v for k,v in epoch_data.items() if k in epoch_keys})
        records.append(record)
df = pd.DataFrame.from_records(records)
print("Shape:", df.shape)

xc_path = Path("excels", name_filter+"_full-report.xlsx" )
df.to_excel(xc_path, index=False)
df.sort_values("eval_f1", ascending=False) #.to_excel("tsa-0823/0823-finetuning_tsa.xlsx")


Shape: (288, 20)


,timestamp,task,machinery,best_epoch,config-file,model_name_or_path,dataset_name,seed,per_device_train_batch_size,task_name,output_dir,num_train_epochs,learning_rate,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,epoch,step
240,01121628,tsa-bin,lumi,15,01121628_tsa-bin_NB-BERT_large_04.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01121628_tsa-bin_NB-BERT_large,/scratch/project_465000144/egilron/sq_label/01...,16,0.00005,0.335106,0.538126,0.563284,0.550418,0.956226,15.0,2025
241,01121628,tsa-bin,lumi,15,01121628_tsa-bin_NB-BERT_large_04.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01121628_tsa-bin_NB-BERT_large,/scratch/project_465000144/egilron/sq_label/01...,16,0.00005,0.331338,0.536876,0.564424,0.550306,0.955725,14.0,1890
242,01121628,tsa-bin,lumi,15,01121628_tsa-bin_NB-BERT_large_04.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01121628_tsa-bin_NB-BERT_large,/scratch/project_465000144/egilron/sq_label/01...,16,0.00005,0.338797,0.538798,0.562144,0.550223,0.956149,16.0,2160
243,01121628,tsa-bin,lumi,15,01121628_tsa-bin_NB-BERT_large_04.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01121628_tsa-bin_NB-BERT_large,/scratch/project_465000144/egilron/sq_label/01...,16,0.00005,0.229123,0.529787,0.567845,0.548156,0.956111,6.0,810
244,01121628,tsa-bin,lumi,15,01121628_tsa-bin_NB-BERT_large_04.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01121628_tsa-bin_NB-BERT_large,/scratch/project_465000144/egilron/sq_label/01...,16,0.00005,0.250492,0.510848,0.590650,0.547858,0.954335,7.0,945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_07.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,0.00001,NaN,0.000000,0.000000,0.000000,0.934185,11.0,1485
123,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_07.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,0.00001,NaN,0.000000,0.000000,0.000000,0.934185,12.0,1620
124,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_07.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,0.00001,NaN,0.000000,0.000000,0.000000,0.934185,13.0,1755
125,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_07.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,0.00001,NaN,0.000000,0.000000,0.000000,0.934185,14.0,1890


Report hyperparameters
cols = ["timestamp",   "machinery", 'model_name_or_path', 'task_name', 'dataset_name', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
records = []
for col in cols:
    record = {col: df[col].unique()}

In [53]:
cols = ["timestamp",   "machinery", 'model_name_or_path','dataset_name', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
records = {}
longest = 0
for col in cols:
    records[col]= df[col].unique()
    longest = max(longest, len(records[col]))
df_vars = pd.DataFrame(columns=cols)
for i in range(longest):
    for col in cols:
        try:
            df_vars.at[i, col] =records[col][i]
            if col=="learning_rate":
                df_vars.at[i, col] =f"{records[col][i]}"

        except:
            df_vars.at[i, col] =""
df_vars


,timestamp,machinery,model_name_or_path,dataset_name,seed,per_device_train_batch_size,learning_rate,num_train_epochs
0,01121628,lumi,ltg/norbert3-large,data/tsa_binary,101,64,5e-05,16
1,,,ltg/norbert3-base,,,,1e-06,
2,,,xlm-roberta-base,,,,1e-05,
3,,,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,,,,,
4,,,ltg/norbert3-small,,,,,
5,,,NbAiLab/nb-bert-large,,,,,


In [54]:
# Best epochs only
df_bests = df.loc[ df["epoch"] ==df["best_epoch"]].copy()
df_bests["learning_rate"] = df_bests["learning_rate"].apply(lambda x: f"{x:.0e}")
report = df_bests[['timestamp', 'task', 'machinery', 'best_epoch', 
       'model_name_or_path',
       'per_device_train_batch_size',  
      'learning_rate', 'eval_loss', 'eval_precision',
       'eval_recall', 'eval_f1', 'eval_accuracy',  ]].sort_values("eval_f1", ascending=False)
xc_path = Path("excels", name_filter+"_best-report.xlsx" )
report.to_excel(xc_path, index=False)
report

,timestamp,task,machinery,best_epoch,model_name_or_path,per_device_train_batch_size,learning_rate,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy
240,01121628,tsa-bin,lumi,15,NbAiLab/nb-bert-large,64,5e-05,0.335106,0.538126,0.563284,0.550418,0.956226
224,01121628,tsa-bin,lumi,9,NbAiLab/nb-bert-large,64,1e-05,0.193528,0.522388,0.558723,0.539945,0.954412
192,01121628,tsa-bin,lumi,16,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,64,5e-05,0.339139,0.476684,0.524515,0.499457,0.947541
48,01121628,tsa-bin,lumi,13,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,64,1e-05,0.198927,0.479915,0.517674,0.498080,0.948661
32,01121628,tsa-bin,lumi,13,xlm-roberta-base,64,1e-05,0.200411,0.470779,0.496009,0.483065,0.947232
96,01121628,tsa-bin,lumi,16,xlm-roberta-base,64,5e-05,0.355596,0.434739,0.493729,0.462360,0.941674
160,01121628,tsa-bin,lumi,16,NbAiLab/nb-bert-large,64,1e-06,0.162107,0.417401,0.432155,0.424650,0.948275
64,01121628,tsa-bin,lumi,16,ltg/norbert3-small,64,1e-05,0.226876,0.336746,0.337514,0.337130,0.940902
144,01121628,tsa-bin,lumi,16,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,64,1e-06,0.195129,0.381513,0.258837,0.308424,0.940979
128,01121628,tsa-bin,lumi,16,xlm-roberta-base,64,1e-06,0.207057,0.351429,0.140251,0.200489,0.939319


In [55]:
a = 0.001
f"{a:.1e}"
df_bests

,timestamp,task,machinery,best_epoch,config-file,model_name_or_path,dataset_name,seed,per_device_train_batch_size,task_name,output_dir,num_train_epochs,learning_rate,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,epoch,step
0,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_large_05.json,ltg/norbert3-large,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_large,/scratch/project_465000144/egilron/sq_label/01...,16,5e-05,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
16,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_13.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-06,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
32,01121628,tsa-bin,lumi,13,01121628_tsa-bin_XLM-R_base_08.json,xlm-roberta-base,data/tsa_binary,101,64,01121628_tsa-bin_XLM-R_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,0.200411,0.470779,0.496009,0.483065,0.947232,13.0,1755
48,01121628,tsa-bin,lumi,13,01121628_tsa-bin_NB-Roberta_base_09.json,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,data/tsa_binary,101,64,01121628_tsa-bin_NB-Roberta_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,0.198927,0.479915,0.517674,0.498080,0.948661,13.0,1755
64,01121628,tsa-bin,lumi,16,01121628_tsa-bin_NorBERT_3_small_06.json,ltg/norbert3-small,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_small,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,0.226876,0.336746,0.337514,0.337130,0.940902,16.0,2160
80,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_01.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,5e-05,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
96,01121628,tsa-bin,lumi,16,01121628_tsa-bin_XLM-R_base_02.json,xlm-roberta-base,data/tsa_binary,101,64,01121628_tsa-bin_XLM-R_base,/scratch/project_465000144/egilron/sq_label/01...,16,5e-05,0.355596,0.434739,0.493729,0.462360,0.941674,16.0,2160
112,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_07.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
128,01121628,tsa-bin,lumi,16,01121628_tsa-bin_XLM-R_base_14.json,xlm-roberta-base,data/tsa_binary,101,64,01121628_tsa-bin_XLM-R_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-06,0.207057,0.351429,0.140251,0.200489,0.939319,16.0,2160
144,01121628,tsa-bin,lumi,16,01121628_tsa-bin_NB-Roberta_base_15.json,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,data/tsa_binary,101,64,01121628_tsa-bin_NB-Roberta_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-06,0.195129,0.381513,0.258837,0.308424,0.940979,16.0,2160
